In [1]:
import torch
from torch_geometric.data import Data
from torch_geometric.data import HeteroData
from hgt_conv_edge_attr import HGTConv
from torch_geometric.nn import HANConv,SAGEConv,to_hetero, Linear
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader
from torch_geometric.data import Dataset
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
import torch_geometric.transforms as T
from torch.utils.data import ConcatDataset

import matplotlib.pyplot as plt

In [2]:
class CustomDataset(Dataset):
    """自定义的异构图数据集类"""
    def __init__(self, root, transform=None, pre_transform=None, indices=None):
        super().__init__(root, transform, pre_transform)
        self.file_list = [f for f in os.listdir(root) if f.endswith('.pt')]
        
        if indices is not None:
            # 如果提供了索引，根据索引筛选文件
            self.file_list = [self.file_list[i] for i in indices]

    def len(self):
        """返回数据集中的图数量"""
        return len(self.file_list)

    def get(self, idx):
        """加载并返回指定索引处的图"""
        file_path = os.path.join(self.root, self.file_list[idx])
        data = torch.load(file_path)
        return data
    
    @property
    def processed_file_names(self):
        """返回处理后的文件名列表"""
        return self.filenames


# 数据集文件夹路径
root_path = '/home/QFRC/ZQS/Projects/Graph/Scripts_demo/Graphs_demos/Graphs3days_all1.0'

# 创建数据集实例
dataset = CustomDataset(root=root_path)
batch_size= 512
# 数据集的长度
dataset_length = len(dataset)

# 计算各个子集的索引范围
train_size = int(0.7 * dataset_length)
val_size = int(0.2 * dataset_length)
test_size = dataset_length - train_size - val_size  # 剩下的部分为测试集

# 生成顺序索引
indices = list(range(dataset_length))

# 根据比例按顺序切分数据集
train_indices = indices[:train_size]  # 前70%的索引
val_indices = indices[train_size:train_size + val_size]  # 接下来的20%
test_indices = indices[train_size + val_size:]  # 最后的10%

# 根据索引创建数据集的子集
train_dataset = CustomDataset(root=root_path, indices=train_indices)
val_dataset = CustomDataset(root=root_path, indices=val_indices)
test_dataset = CustomDataset(root=root_path, indices=test_indices)

# 创建对应的DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [3]:
# 3days

print(val_indices)
label_indices_pre=[x + 1 for x in val_indices]
open_price=round(dataset[542]['stock'].x[0,10].item(), 2)
close_price=round(dataset[542]['stock'].x[0,13].item(), 2)
print(open_price)
print(close_price)

print(label_indices_pre)

print(dataset[0]['stock'].x)
print(type(dataset[0]['stock'].x))
print(dataset[0]['stock'].x[0,0])
print(type(dataset[0]['stock'].x[0,0]))

float_value=round(dataset[0]['stock'].x[0,0].item(), 2)
print(float_value)
print(round(dataset[0]['stock'].x[0,10].item(), 2))



[543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697]
38.05
41.5
[544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 58

In [ ]:
# 5days

print(val_indices)
label_indices_pre=[x + 1 for x in val_indices]
open_price=round(dataset[542]['stock'].x[0,20].item(), 2)
close_price=round(dataset[542]['stock'].x[0,23].item(), 2)
print(open_price)
print(close_price)

print(label_indices_pre)

print(dataset[0]['stock'].x)
print(type(dataset[0]['stock'].x))
print(dataset[0]['stock'].x[0,0])
print(type(dataset[0]['stock'].x[0,0]))

float_value=round(dataset[0]['stock'].x[0,0].item(), 2)
print(float_value)
print(round(dataset[0]['stock'].x[0,20].item(), 2))



In [4]:
# 2023-4-10
# 新的列表，用于存储提取后的值
open_price_pre = []  # 预测天的开盘价
close_price_pre = []  # 预测天的收盘价
open_price_today=[]  # 当天开盘价
close_price_today=[]


# 循环遍历每个索引
for i in label_indices_pre:
    # 提取第 i 个数据集中 'stock' 的 (0,0) 位置的值
    open_value = round(dataset[i]['stock'].x[0, 0].item(),2)  # 提取并转换为 float
    open_price_pre.append(open_value)  # 保存到新的列表中
    
    close_value = round(dataset[i]['stock'].x[0, 3].item(),2)  # 提取并转换为 float
    close_price_pre.append(close_value)  # 保存到新的列表中

open_price_today=open_price_pre    
print(open_price_today)
open_price_today.insert(0, open_price)
print(open_price_today)

close_price_today=close_price_pre
close_price_today.insert(0,close_price)
print(close_price_today)

# 输出新的列表
print(open_price_pre)
print(close_price_pre)

print(open_price_today)
print(close_price_today)


[38.05, 42.0, 43.0, 39.39, 38.51, 39.7, 41.3, 42.1, 41.9, 43.0, 46.0, 46.2, 43.5, 38.52, 40.6, 41.5, 38.78, 38.01, 39.0, 36.3, 35.34, 35.78, 35.9, 35.6, 37.36, 39.95, 39.08, 38.5, 36.92, 39.8, 40.45, 41.0, 39.45, 44.5, 43.55, 45.28, 44.08, 44.75, 43.47, 42.62, 42.67, 45.41, 45.0, 48.99, 52.2, 51.7, 58.21, 62.15, 62.6, 59.61, 57.0, 50.73, 47.07, 46.97, 48.0, 47.61, 47.37, 45.9, 47.78, 46.05, 45.0, 42.58, 41.2, 42.75, 44.25, 47.56, 47.0, 44.89, 43.05, 42.4, 46.01, 46.68, 44.4, 45.0, 46.58, 45.08, 47.07, 46.18, 47.2, 51.7, 49.91, 48.72, 46.3, 47.56, 44.88, 47.47, 43.49, 42.15, 42.67, 40.86, 41.62, 42.03, 42.0, 40.3, 39.28, 36.92, 39.1, 38.9, 39.0, 38.85, 38.86, 37.65, 37.37, 36.22, 35.38, 37.5, 37.9, 37.7, 37.66, 35.54, 35.5, 35.28, 34.48, 36.4, 37.39, 36.56, 36.7, 36.65, 37.0, 37.6, 36.85, 37.94, 36.5, 35.24, 35.0, 32.32, 30.0, 30.8, 29.31, 28.6, 28.5, 27.98, 29.03, 29.0, 29.75, 30.18, 30.1, 29.11, 30.3, 30.88, 31.5, 31.73, 34.1, 34.44, 34.75, 35.35, 34.2, 34.22, 35.25, 37.69, 36.28, 35.

In [ ]:
class HGTExplicit(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels1, hidden_channels2, hidden_channels3, out_channels1, out_channels2, num_heads, metadata1, metadata2):
        super().__init__()

        # 定义两层 HGT 卷积
        
        self.conv1 = HGTConv(in_channels, hidden_channels1, metadata1, num_heads)
        self.conv2 = HGTConv(hidden_channels1, hidden_channels2, metadata2, num_heads)
        self.conv3 = HGTConv(hidden_channels2, hidden_channels3, metadata2, num_heads)
        # self.conv4 = HGTConv(hidden_channels3, hidden_channels4, metadata2, num_heads)

        # 用于将 connect 节点特征从 5 维转换为 hidden_channels1
        self.connect_lin1 = torch.nn.Linear(5, hidden_channels1)
        self.connect_lin2 = torch.nn.Linear(5, hidden_channels2)
        self.connect_lin3 = torch.nn.Linear(5, hidden_channels3)
        
        self.fs_lin1= torch.nn.Linear(2, hidden_channels1)
        # 用于将 connect 节点特征从 hidden_channels1 转换为 hidden_channels2
        self.fs_lin2 = torch.nn.Linear(2, hidden_channels2)
        self.fs_lin3 = torch.nn.Linear(2, hidden_channels3)
        
        # Dropout 层
        self.dropout = torch.nn.Dropout(p=0.5)
        
        # 输出层
        self.gru=torch.nn.GRU(hidden_channels3, out_channels1)
        self.lstm=torch.nn.LSTM(hidden_channels3, out_channels1)
        #  self._initialize_weights()  # 初始化权重
        
        self.out1 = Linear(hidden_channels3, out_channels1)
        self.out2 = Linear(out_channels1, out_channels2)
        # self.out3 = Linear(out_channels2, out_channels3)
        
        # 初始化 GRU 权重
        # self._initialize_weights()
        # 定义 PyG 的特征归一化变换
        self.normalize = T.NormalizeFeatures()
        

    def forward(self, data):
        
        
        # 对整个异构图进行归一化
        # data = self.normalize(data)
        # 获取 x_dict, edge_index_dict, edge_attr_dict
        x_dict, edge_index_dict, edge_attr_dict = data.x_dict, data.edge_index_dict, data.edge_attr_dict

        
        connect_initial = x_dict['connect']
        financing_initial = x_dict['financing']
        selling_initial = x_dict['selling']

        x_dict = self.conv1(x_dict, edge_index_dict, edge_attr_dict)
        x_dict = {node_type: F.leaky_relu(x) for node_type, x in x_dict.items()}
        # 应用Dropout
        x_dict = {node_type: self.dropout(x) for node_type, x in x_dict.items()} #神经元已经提取了特征，再对这些特征进行随机“丢弃”可以更好地防止过拟合。
               
        # 移除不再使用的节点
        for key in ['connect', 'financing', 'selling']:
            if key in x_dict:
                del x_dict[key]

        for edge_type in [('connect', 'invest', 'stock'), ('financing', 'invest', 'stock'), ('selling', 'invest', 'stock')]:
            if edge_type in edge_index_dict:
                del edge_index_dict[edge_type]
            if edge_type in edge_attr_dict:
                del edge_attr_dict[edge_type]
        
        # 第二层 HGT 卷积
        x_dict = self.conv2(x_dict, edge_index_dict, edge_attr_dict)
        x_dict = {node_type: F.leaky_relu(x) for node_type, x in x_dict.items()}
        x_dict = {node_type: self.dropout(x) for node_type, x in x_dict.items()} 

        # 第三层 HGT 卷积
        x_dict = self.conv3(x_dict, edge_index_dict,edge_attr_dict)
        x_dict = {node_type: F.leaky_relu(x) for node_type, x in x_dict.items()}
        x_dict = {node_type: self.dropout(x) for node_type, x in x_dict.items()} 
        
        
        
        out=F.leaky_relu(self.out1(x_dict['stock']))
        out=self.out2(out)[::12]

        return out
    
# 传参        
in_channels = {
    'stock': 15,
    'other': 768,
    'connect': 3,
    'financing': 2,
    'selling': 2
}

metadata1 = (['stock', 'other', 'connect', 'financing', 'selling'], [
    ('stock', 'spearman', 'stock'),
    ('connect', 'invest', 'stock'),
    ('financing', 'invest', 'stock'),
    ('selling', 'invest', 'stock'),
    ('stock', 'relationship', 'stock'),
    ('stock', 'relationship', 'other'),
    ('other', 'relationship', 'stock'),
    ('other', 'relationship', 'other')
])

metadata2 = (['stock', 'other',], [
    ('stock', 'spearman', 'stock'),
    ('stock', 'relationship', 'stock'),
    ('stock', 'relationship', 'other'),
    ('other', 'relationship', 'stock'),
    ('other', 'relationship', 'other')
])

hidden_channels1 = 256
hidden_channels2 = 32
hidden_channels3 = 32
out_channels1 = 32
out_channels2 = 2

num_heads = 32



In [ ]:
model = HGTExplicit(in_channels, hidden_channels1, hidden_channels2, hidden_channels3, out_channels1, out_channels2, num_heads, metadata1, metadata2)
model.load_state_dict(torch.load('/home/QFRC/ZQS/Projects/Graph/Scripts_demo/Improve/3days/Grid_saves_顺序/models/256_32_32_32/ACC_0.5548_MCC_0.2044_Precision_0.5221_Recall_0.9467_F1_0.6730_AUC_0.5702_133265.pth'))  # 加载保存的模型权重
model.eval()  # 设置为评估模式


In [ ]:
criterion = torch.nn.CrossEntropyLoss()
total_loss = 0
labels = []
preds = []
probs = []

with torch.no_grad():
    for data in (val_loader):
        
        y = data['stock'].y[:, 0].to(dtype=torch.long)  # (batch_size, num_stocks) -> (num_stocks, batch_size)
        
        output = model(data)  # 输出形状: (num_stocks, batch_size)
        # print(output)
        loss = criterion(output, y)
        total_loss += loss.item()
        
            # 获取预测概率
        probs_batch = torch.softmax(output, dim=1)[:, 1]  # 只取正类的概率
        
        # 将logits转为概率并转为二元预测
        preds_batch = torch.argmax(output, dim=1)  # 获取类别预测
        
        labels.extend(y.cpu().numpy())
        preds.extend(preds_batch.cpu().numpy())
        probs.extend(probs_batch.cpu().numpy())
        

# 计算评价指标
accuracy = accuracy_score(labels, preds)
precision = precision_score(labels, preds, zero_division=1)
recall = recall_score(labels, preds, zero_division=1)
f1 = f1_score(labels, preds, zero_division=1)
mcc= matthews_corrcoef(labels, preds)

# 计算AUC
try:
    auc = roc_auc_score(labels, probs)
except ValueError:
    auc = 0  # 如果计算AUC时遇到问题，例如只有一种类标签

print(f'Accuracy: {accuracy:.4f}, Mcc: {mcc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}, AUC: {auc:.4f}')
print(preds)

In [ ]:
print(preds)
print(y)
print(len(preds))
print(len(y))
print(type(preds))
print(type(y))

In [ ]:
# 模拟投资
# # 4-9准备投资，预测出的涨跌是4-10~11-27
# preds：预测一共155个交易日收盘价的涨跌
# open_price_today：155个交易日的开盘价和前一天的开盘价
# open_price_pre：155个交易日的开盘价
# close_price_today：155天个交易日收盘价和前一天的开盘价
# close_price_pre：155个交易日的收盘价

# 逻辑：预测下一交易日上涨，则下一交易日开盘价买入


# 初始资金
initial_capital = 100000  # 10 万元初始资金
capital = initial_capital
position = 0  # 初始没有持仓
cash = capital

# 记录每日资产
capital_history = []

# 记录买入的标志
bought = False
        

# 回测逻辑，处理前154天
for i in range(len(preds) - 1):  # 循环到倒数第二天，最后一天单独处理
    if not bought and preds[i] == 1:  # 如果还未买入且预测当天上涨
        # 预测上涨，所以以当天的开盘价买入
        position = cash // open_price_today[i]  # 以当天的开盘价买入
        cash -= position * open_price_today[i]  # 扣除买入股票的花费
        print(f"第{i}天：开盘价 {open_price_today[i]} 买入 {position} 股，剩余现金 {cash:.2f}")
        bought = True
    elif bought and preds[i] == 0:  # 如果已经持仓且预测当天下跌，则当天开盘卖出
        cash += position * open_price_today[i]  # 以当天的开盘价卖出
        print(f"第{i}天：以开盘价 {open_price_today[i]} 卖出 {position} 股，现金 {cash:.2f}")
        position = 0  # 清空持仓
        bought = False
        
    # 记录每日的总资金（现金 + 持仓市值）
    if bought:
        total_assets = cash + position * close_price_today[i]  # 现金 + 持仓股票市值（按收盘价计算）
    else:
        total_assets = cash  # 只有现金，没有持仓

    capital_history.append(round(total_assets,2))  # 记录每日的总资产

        
# 调试输出，检查最后一天前是否持有股票
print(f"最后一个交易日前，持仓股数: {position}, 预测结果: {preds[-1]}")

# 特殊处理最后一个交易日（第155天）
if preds[-1] == 1:  # 如果预测最后一天（第155天）上涨
    if position > 0:
        cash += position * close_price_today[-1]  # 以最后一天的收盘价卖出
        print(f"最后一天：收盘价 {close_price_today[-1]} 卖出 {position} 股，现金 {cash:.2f}")
        position = 0  # 清空持仓
    else:
        print("最后一天预测上涨，但没有持仓，无法卖出")
elif preds[-1] == 0:  # 如果预测最后一天（第155天）下跌
    if position > 0:
        cash += position * open_price_today[-1]  # 以最后一天的开盘价卖出
        print(f"最后一天：开盘价 {open_price_today[-1]} 卖出 {position} 股，现金 {cash:.2f}")
        position = 0  # 清空持仓
    else:
        print("最后一天预测下跌，但没有持仓，无法卖出")
else:
    print("预测结果没有定义")
    
# 最后一天的总资金记录
capital_history.append(round(cash,2))  # 无论是否有持仓，最后一天全部资金都变为现金    
ration_capital= [(capital_history[i] - initial_capital) / initial_capital * 100 for i in range(1, len(capital_history))]
    
print(capital_history)
print(ration_capital)
        


In [ ]:
# 买入并持有策略逻辑
buy_and_hold_history = []

# 第一天开盘价买入所有股票
buy_and_hold_position = initial_capital // open_price_today[0]
buy_and_hold_cash = initial_capital - buy_and_hold_position * open_price_today[0]

# 计算每一天的总资金（持有股票市值 + 现金）
for i in range(len(close_price_today)):
    buy_and_hold_assets = buy_and_hold_cash + buy_and_hold_position * close_price_today[i]  # 现金 + 股票市值
    buy_and_hold_history.append(buy_and_hold_assets)

# 最后一天全部持仓卖出
buy_and_hold_cash += buy_and_hold_position * close_price_today[-1]
buy_and_hold_history.append(buy_and_hold_cash)

ration_buy_and_hold= [(buy_and_hold_history[i] - initial_capital) / initial_capital * 100 for i in range(1, len(capital_history))]


print(buy_and_hold_history)
print(ration_buy_and_hold)

In [ ]:
# 绘制资金变化折线图
plt.figure(figsize=(10, 6))
plt.plot(capital_history, label="MT",  linestyle='-', color='#B19CD9')
plt.plot(buy_and_hold_history, label="B&H", linestyle='-', color='#FFD580')
plt.title("Accumulate Capital", fontsize=16)
plt.xlabel("Trading day", fontsize=12)
plt.ylabel("RMB (¥)", fontsize=12)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()